In [73]:
import pandas as pd
import func
from tqdm import tqdm
tqdm.pandas()

In [72]:
# Readfile
df = pd.read_excel('d_item 20220916.xlsx', sheet_name='Sheet5')
# Kolom yang digunakan
df = df[['kdsatker', 'nmitem', 'volkeg', 'jumlah']]

In [74]:
# Keyword
df_keyword = pd.read_excel('Daftar Keyword Penyedia VS Non Penyedia.xlsx')

# Keyword per masing-masing klasifikasi
keyword_penyedia = df_keyword['Penyedia'].dropna().drop_duplicates().str.lower().to_list()
keyword_non_penyedia = df_keyword['Non Penyedia'].dropna().drop_duplicates().str.lower().to_list()
keyword_selfblocking = df_keyword['Selfblocking'].dropna().drop_duplicates().str.lower().to_list()

In [76]:
# Fungsi get klasifikasi

def get_klasifikasi(teks):
    teks = teks.lower()

    # Cek Keyword Penyedia
    penyedia = 0
    for keyword in keyword_penyedia:
        if keyword in teks:
            penyedia = penyedia + 1

    # Cek Keyword Non-Penyedia
    non_penyedia = 0
    for keyword in keyword_non_penyedia:
        if keyword in teks:
            non_penyedia = non_penyedia + 1

    # Cek Keyword selfblocking
    selfblocking = 0
    for keyword in keyword_selfblocking:
        if keyword in teks:
            selfblocking = selfblocking + 1

    list_nilai = [penyedia, non_penyedia, selfblocking]

    if list_nilai == [0,0,0]:
        klasifikasi = '-'
    else:
        # Variabel terbesar
        max_value = max(list_nilai)

        pil = list_nilai.index(max_value)

        if pil == 0:
            klasifikasi = 'Penyedia'
        elif pil == 1:
            klasifikasi = 'Non-Penyedia'
        elif pil == 2:
            klasifikasi = 'Selfblocking'

    # print(f'{list_nilai} - {klasifikasi}')
    return klasifikasi

In [77]:
# Get kolom klasifikasi
df['klasifikasi'] = df['nmitem'].progress_apply(get_klasifikasi)

100%|██████████| 238758/238758 [00:10<00:00, 22291.63it/s]


In [78]:
df[df['klasifikasi'] != '-']

,kdsatker,nmitem,volkeg,jumlah,klasifikasi
0,18576,honor tim wakil ketua,25,15000000,Non-Penyedia
1,18576,honor tim anggota,152,76000000,Non-Penyedia
2,18576,pengadaan komputer supplies,1,100000000,Penyedia
3,18576,honor narasumber eselon i,2,2800000,Non-Penyedia
4,18576,perjalanan dinas supervisi,15,101889000,Non-Penyedia
...,...,...,...,...,...
238753,690332,honor pembuat soal,400,40000000,Non-Penyedia
238754,690332,honor telaah bahasa soal,400,8000000,Non-Penyedia
238755,690332,honor telaah materi soal,400,18000000,Non-Penyedia
238756,690332,honor tim,179,89500000,Non-Penyedia


In [114]:
# GroupBy kd_satker dan klasifikasi
df_group = df.groupby(['kdsatker', 'klasifikasi']).agg(jumlah_kegiatan = ('volkeg','sum'), jumlah_pagu = ('jumlah','sum')).astype('Int64').reset_index()
df_group

,kdsatker,klasifikasi,jumlah_kegiatan,jumlah_pagu
0,18576,-,133103,319076990000
1,18576,Non-Penyedia,44765,95171721000
2,18576,Penyedia,23159147,370724581000
3,18576,Selfblocking,4,48842211000
4,18580,-,190,17446993000
...,...,...,...,...
1755,690328,Penyedia,139594,26891189000
1756,690328,Selfblocking,1,352746000
1757,690332,-,61117,26823410000
1758,690332,Non-Penyedia,104042,9592473000


In [119]:
# DF Jumlah kegiatan
df_jumlah = df_group.set_index(['kdsatker', 'klasifikasi'])['jumlah_kegiatan'].unstack().reset_index()
df_jumlah = df_jumlah[['kdsatker','Non-Penyedia', 'Penyedia', 'Selfblocking']]
df_jumlah = df_jumlah.rename(columns={'kdsatker':'Kode Satker', 'Non-Penyedia':'Non-Penyedia (Jumlah)', 'Penyedia':'Penyedia (Jumlah)', 'Selfblocking':'Selfblocking (Jumlah)'})

df_jumlah

klasifikasi,Kode Satker,Non-Penyedia (Jumlah),Penyedia (Jumlah),Selfblocking (Jumlah)
0,18576,44765,23159147,4
1,18580,3627,1855048,127
2,18597,6343,9103188,77632
3,18602,67786,2087639,63
4,18619,86393,3275285,115
...,...,...,...,...
512,689181,8371,75366,<NA>
513,689198,21810,125339,<NA>
514,689203,9086,49772,<NA>
515,690328,19125,139594,1


In [120]:
# DF Jumlah anggaran
df_anggaran = df_group.set_index(['kdsatker', 'klasifikasi'])['jumlah_pagu'].unstack().reset_index()
df_anggaran = df_anggaran[['kdsatker','Non-Penyedia', 'Penyedia', 'Selfblocking']]
df_anggaran = df_anggaran.rename(columns={'kdsatker':'Kode Satker', 'Non-Penyedia':'Non-Penyedia (Pagu)', 'Penyedia':'Penyedia (Pagu)', 'Selfblocking':'Selfblocking (Pagu)'})

df_anggaran

klasifikasi,Kode Satker,Non-Penyedia (Pagu),Penyedia (Pagu),Selfblocking (Pagu)
0,18576,95171721000,370724581000,48842211000
1,18580,1684084000,8586637000,1057276000
2,18597,4208557000,31827823000,2861308000
3,18602,23592836000,13080724000,1041106000
4,18619,33841156000,18021072000,1562376000
...,...,...,...,...
512,689181,1687748000,1106907000,<NA>
513,689198,2755360000,2602812000,<NA>
514,689203,2125498000,1292939000,<NA>
515,690328,15320197000,26891189000,352746000


In [121]:
df_jumlah.merge(df_anggaran, how='left', on='Kode Satker')

klasifikasi,Kode Satker,Non-Penyedia (Jumlah),Penyedia (Jumlah),Selfblocking (Jumlah),Non-Penyedia (Pagu),Penyedia (Pagu),Selfblocking (Pagu)
0,18576,44765,23159147,4,95171721000,370724581000,48842211000
1,18580,3627,1855048,127,1684084000,8586637000,1057276000
2,18597,6343,9103188,77632,4208557000,31827823000,2861308000
3,18602,67786,2087639,63,23592836000,13080724000,1041106000
4,18619,86393,3275285,115,33841156000,18021072000,1562376000
...,...,...,...,...,...,...,...
512,689181,8371,75366,<NA>,1687748000,1106907000,<NA>
513,689198,21810,125339,<NA>,2755360000,2602812000,<NA>
514,689203,9086,49772,<NA>,2125498000,1292939000,<NA>
515,690328,19125,139594,1,15320197000,26891189000,352746000


In [102]:
df_group.pivot('kdsatker','klasifikasi','jumlah_kegiatan').reset_index(drop=True)

klasifikasi,-,Non-Penyedia,Penyedia,Selfblocking
0,133103,44765,23159147,4
1,190,3627,1855048,127
2,8383409,6343,9103188,77632
3,4097,67786,2087639,63
4,6068,86393,3275285,115
...,...,...,...,...
512,518,8371,75366,<NA>
513,333,21810,125339,<NA>
514,236,9086,49772,<NA>
515,15765,19125,139594,1
